# Model Management

The Model Management module in Dataloop provides a comprehensive solution for managing the entire lifecycle of your AI models. With this module, you have the flexibility to either use pre-existing public AI models or connect and utilize your own custom models.

You can train different versions of your models using data Items and their Annotations, experiment with various hyper-parameters to find the best configuration, and evaluate your models by inferencing over Datasets. The module also allows you to compare the training and evaluation metrics of your models, review false-positive and false-negative instances, and make informed decisions based on the results.

Once you have trained and evaluated your models, you can deploy them as a service within Dataloop applications. This enables you to run your models at scale and make them available to a wider audience. Additionally, the Model-Management module provides the tools to build continuous learning pipelines, where your models can continuously learn and improve over time.


## Why use Dataloop's model management feature?

Dataloop's model management feature enables machine learning engineers to centralize their AI model research, training, and production processes. The models are executed using Dpks, Datasets, and Artifacts. The code required to run the model is contained in Dpk and pushed to the Dataloop cloud-native SaaS platform. The Datasets hold the images used for training or inference and define which images belong to each subset, such as train/validation/test. Models can be obtained from ready-to-use open source algorithms or created from pre-trained models for further fine-tuning or transfer learning. Additionally, one can upload their own models and evaluate performance by viewing training metrics. All models can be incorporated into the Dataloop platform and integrated into the UI through buttons or slots, or added to Pipelines.

The Model Comparison feature in the Dataloop platform enables users to view and compare different versions of models all in one place. This can be done by selecting user-specified metrics to evaluate the models.

The Model Management system in the platform can work in two ways: "offline" mode and "online" mode.

* In the **offline mode**, users can train and run models locally using their own data and compare the configurations and metrics of different models on the platform. This mode does not require extensive platform integration and can be used after creating dl.Dpk and dl.Model entities. However, it is limited to visualizing metrics that have been exported from the local model training sessions only. The code and weights of the models are not saved in the Dataloop platform, but the metrics can be viewed at a later time.

* On the other hand, the **online mode** allows users to train models that can be deployed anywhere within the platform. For instance, users can create a button interface to use their model for inferencing on new data and view the results on the platform. To use this mode, users must create a ModelAdapter class and implement the required functions to access the Dataloop API. This mode includes all the features available in the "offline" mode.

## Model Management in Dataloop's Python SDK

Model Management is also available both in the UI web version of the Dataloop platform and in the Dataloop Python SDK.

You can find more about how to use models in our [documentetaion on Model Management](https://developers.dataloop.ai/tutorials/model_management/) or [our Python SDK-focused documentation](https://sdk-docs.dataloop.ai/en/latest/repositories.html#module-dtlpy.repositories.models).

Additionally, here are some code snippets that may be useful:

### 0. [Prerequisite] Install a dataset and a model from an existing dpk on the project

Get the current project

In [ ]:
import dtlpy as dl

project_id = 'project_id'

project = dl.projects.get(project_id=project_id)

Install the app with the dataset and the model

In [ ]:
dpk = project.dpks.get(dpk_name="ml-compare-example")
app = project.apps.install(dpk=dpk)

Get the dataset after the app is installed

In [ ]:
dataset = project.datasets.get(dataset_name="V2 Plant Seedlings Dataset")

### 1. Model Get

In [ ]:
model = project.models.get(model_name='pretrained-resnet')

### 2. Model Clone

To create a new copy of our existing model, run the following command

In [ ]:
clone_model = model.clone(model_name='my pretrained resnet',
                          description='pretrained cloned into my project',
                          project_id=project_id)

### 3. Model Deploy

In case that the model is already ready for production, deploy it by the following command

In [ ]:
model.deploy()

### 4. Model Train

Notice: Deployed models cannot be trained, so in this step and the next ones, the clone model that was created on [step 2: clone a model](model_management.ipynb#2-clone-a-model) will be used.

To train a model on a dataset we need to:
1. Connect a dataset to the model, that will be used for the training.
2. Add the train and validation filters to the model, that will help to split the data to train and validation dataloaders.

In [ ]:
train_filters = dl.Filters(field="metadata.system.tags.train", values=True)
validation_filters = dl.Filters(field="metadata.system.tags.validation", values=True)

clone_model.configuration['id_to_label_map'] = None
clone_model.configuration['label_to_id_map'] = None
clone_model.dataset_id = dataset.id
clone_model.ontology_id = dataset.ontologies.list()[0]
clone_model.add_subset(subset_name="train", subset_filter=train_filters)
clone_model.add_subset(subset_name="validation", subset_filter=validation_filters)
clone_model.update(system_metadata=True)

execution = clone_model.train(service_config={
    "name": "resnet-train-evaluate",
    "runtime": {
        "podType": "regular-m",
        "concurrency": 1,
        "runnerImage": "gcr.io/viewo-g/piper/agent/runner/apps/torch-models:0.1.4",
        "autoscaler": {
            "type": "rabbitmq",
            "minReplicas": 0,
            "maxReplicas": 2,
            "queueLength": 100
        }
    }
})

Wait for the training to finish

In [ ]:
execution.wait()

### 5. Model Predict

First, get the spesifc item we want to perfrom prediction with our model

In [ ]:
item_filename = '/Black-grass/6.png'

item = dataset.items.get(filepath=item_filename)

Run the model predict method on the item.

Notice: Since the predict method expect to recive a list of item ids, we will add the item id to a list that will be sent to the method.

In [ ]:
clone_model.predict(item_ids=[item.id])

### 6. Model Evaluate

Notice: To evaluate a model we need to:
1. Get a dataset, that will be used for the evaluation.
2. Define the test filters, to build the test dataloader.

In [ ]:
test_filters = dl.Filters(field='metadata.system.tags.test', values=True)

clone_model.evaluate(dataset_id=dataset.id,
                     filters=test_filters)